<a href="https://colab.research.google.com/github/sanuthit/Risk-Based-Motor-Insurance-Premium-Calculation-System-/blob/risk-model-development/accident_risk_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Risk-Based Motor Insurance Premium + Premium Calculation** - Risk Model

In [53]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)

# ---- Models (sklearn) ----
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

# Optional: XGBoost if installed
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except Exception:
    HAS_XGB = False

# Optional: CatBoost if installed
try:
    from catboost import CatBoostClassifier
    HAS_CAT = True
except Exception:
    HAS_CAT = False

In [54]:
import pandas as pd
import os

from google.colab import drive
drive.mount('/content/drive')
root = "/content/drive/MyDrive"
df = pd.read_csv("/content/drive/MyDrive/Data/Datasets/risk_dataset_60000_cleaned.csv", encoding="utf-8")

DATA_PATH = "/content/drive/MyDrive/Data/Datasets/risk_dataset_60000_cleaned.csv"
df = pd.read_csv(DATA_PATH)

print(df.shape)
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(60000, 49)


,policy_id,customer_id,driver_age,driver_gender,driver_occupation,years_of_driving_experience,member_automobile_assoc_ceylon,has_previous_motor_policy,ncb_percentage,accidents_last_3_years,...,had_claim_within_1_year,num_claims_within_1_year,total_claim_amount_within_1_year,hard_flag_blacklist,driver_age_band,vehicle_age_band,risk_exposure_proxy,doc_missing_score,compliance_risk_score,ncb_validity_flag
0,P000001,C00002,62,M,Businessman,4,0,1,35,0,...,0,0,0,0,60+,4–7,Low,0,0,0
1,P000002,C00003,69,F,Businessman,20,0,1,20,0,...,0,0,0,0,60+,13+,Low,0,0,0
2,P000003,C00004,37,M,IT Engineer,13,0,0,0,0,...,0,0,0,0,35–44,13+,Low,0,0,0
3,P000004,C00005,63,F,Businessman,31,0,1,20,1,...,0,0,0,0,60+,13+,High,1,0,0
4,P000005,C00006,21,M,Student,0,0,0,0,1,...,0,0,0,0,18–24,13+,Low,0,0,0


In [68]:
df['driver_age_band'] = df['driver_age_band'].str.replace('â€"', '-', regex=False)
df['vehicle_age_band'] = df['vehicle_age_band'].str.replace('â€"', '-', regex=False)

print("\nFixed encoding - unique values:")
print("Driver age bands:", df['driver_age_band'].unique())
print("Vehicle age bands:", df['vehicle_age_band'].unique())


Fixed encoding - unique values:
Driver age bands: ['60+' '35–44' '18–24' '45–59' '25–34']
Vehicle age bands: ['4–7' '13+' '0–3' '8–12']


1. Define the target

In [69]:
RISK_FEATURES = [
    # Driver risk
    "driver_age",
    "driver_age_band",
    "driver_gender",
    "driver_occupation",
    "years_of_driving_experience",
    "member_automobile_assoc_ceylon",

    # Driving & claim history
    "has_previous_motor_policy",
    "accidents_last_3_years",
    "ncb_percentage",

    # Vehicle risk
    "vehicle_type",
    "vehicle_segment",
    "engine_capacity_cc",
    "fuel_type",
    "vehicle_age_years",
    "vehicle_age_band",
    "has_lpg_conversion",

    # Usage & exposure
    "vehicle_usage_type",
    "risk_exposure_proxy",
    "registration_district",
    "parking_type",

    # Compliance proxy
    "doc_missing_score",
    "compliance_risk_score"
]

TARGET = "had_claim_within_1_year"
if TARGET not in df.columns:
    raise ValueError(f"TARGET='{TARGET}' not found in df.columns")

2. Drop columns

In [70]:
DROP_COLS = []
for c in ["policy_id", "customer_id", "proposal_no"]:
    if c in df.columns:
        DROP_COLS.append(c)

FEATURES = [c for c in df.columns if c not in ([TARGET] + DROP_COLS)]
df_model = df[FEATURES + [TARGET]].copy()

In [71]:
df_risk = df[RISK_FEATURES + [TARGET]].copy()

Data Quality

In [73]:
print("\n" + "="*80)
print("DATA QUALITY CHECKS")
print("="*80)

# Check missing values
missing = df_risk.isnull().sum()
if missing.sum() > 0:
    print("\nMissing values found:")
    print(missing[missing > 0])
else:
    print("\n✓ No missing values")

# Check target balance
target_dist = df_risk[TARGET].value_counts(normalize=True)
print(f"\nTarget distribution:")
print(f"  No claim (0): {target_dist[0]:.2%}")
print(f"  Claim (1):    {target_dist[1]:.2%}")


DATA QUALITY CHECKS

✓ No missing values

Target distribution:
  No claim (0): 74.99%
  Claim (1):    25.01%


In [63]:
#Target balance
df_risk[TARGET].value_counts(normalize=True)

,proportion
had_claim_within_1_year,
0,0.749933
1,0.250067


# 4. Split X and y

In [75]:
X = df_risk[RISK_FEATURES]
y = df_risk[TARGET]

# 70% Train, 15% Val, 15% Test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

print(f"\nSplit sizes:")
print(f"  Train: {X_train.shape[0]:,} samples")
print(f"  Val:   {X_val.shape[0]:,} samples")
print(f"  Test:  {X_test.shape[0]:,} samples")


Split sizes:
  Train: 42,000 samples
  Val:   9,000 samples
  Test:  9,000 samples


01. Encoding pipeline

In [76]:
preprocessor = ColumnTransformer(
    transformers=[
        # Ordinal bands (now with correct hyphens)
        ("ord", OrdinalEncoder(
            categories=[
                ["18-24", "25-34", "35-44", "45-59", "60+"],
                ["0-3", "4-7", "8-12", "13+"]
            ],
            handle_unknown="use_encoded_value",
            unknown_value=-1
        ), ["driver_age_band", "vehicle_age_band"]),

        # Nominal categories
        ("cat", OneHotEncoder(
            handle_unknown="ignore",
            sparse_output=False
        ), [
            "driver_gender",
            "driver_occupation",
            "vehicle_type",
            "vehicle_segment",
            "fuel_type",
            "vehicle_usage_type",
            "risk_exposure_proxy",
            "registration_district",
            "parking_type"
        ]),

        # Numeric/binary features
        ("num", "passthrough", [
            "driver_age",
            "years_of_driving_experience",
            "member_automobile_assoc_ceylon",
            "has_previous_motor_policy",
            "accidents_last_3_years",
            "ncb_percentage",
            "engine_capacity_cc",
            "vehicle_age_years",
            "has_lpg_conversion",
            "doc_missing_score",
            "compliance_risk_score"
        ])
    ]
)

In [77]:
X_train_enc = preprocessor.fit_transform(X_train)
X_val_enc = preprocessor.transform(X_val)
X_test_enc = preprocessor.transform(X_test)

print(f"\nEncoded feature dimensions: {X_train_enc.shape[1]}")


Encoded feature dimensions: 55


CALCULATE CLASS WEIGHTS

In [79]:
from sklearn.utils.class_weight import compute_class_weight

classes = np.array([0, 1])
weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y_train
)

class_weight_dict = {0: weights[0], 1: weights[1]}
print(f"\nClass weights: {class_weight_dict}")


Class weights: {0: np.float64(0.6667301647775978), 1: np.float64(1.9994287346472437)}


3. evaluater

In [80]:
def evaluate_model(y_true, proba, threshold=None, model_name="Model"):
    """Comprehensive model evaluation with optimal threshold finding"""

    # Calculate ROC-AUC and PR-AUC
    roc_auc = roc_auc_score(y_true, proba)
    pr_auc = average_precision_score(y_true, proba)

    # Find optimal threshold if not provided
    if threshold is None:
        thresholds = np.arange(0.05, 0.95, 0.01)
        f1_scores = []
        for t in thresholds:
            pred = (proba >= t).astype(int)
            f1_scores.append(f1_score(y_true, pred, zero_division=0))

        best_idx = np.argmax(f1_scores)
        threshold = thresholds[best_idx]
        best_f1 = f1_scores[best_idx]
    else:
        pred = (proba >= threshold).astype(int)
        best_f1 = f1_score(y_true, pred, zero_division=0)

    # Calculate metrics at best threshold
    pred = (proba >= threshold).astype(int)
    precision = precision_score(y_true, pred, zero_division=0)
    recall = recall_score(y_true, pred, zero_division=0)

    print(f"\n{'='*80}")
    print(f"{model_name} - EVALUATION RESULTS")
    print(f"{'='*80}")
    print(f"ROC-AUC:           {roc_auc:.4f}")
    print(f"PR-AUC:            {pr_auc:.4f}")
    print(f"Optimal Threshold: {threshold:.3f}")
    print(f"F1-Score:          {best_f1:.4f}")
    print(f"Precision:         {precision:.4f}")
    print(f"Recall:            {recall:.4f}")
    print(f"\nConfusion Matrix:")
    print(confusion_matrix(y_true, pred))
    print(f"\nClassification Report:")
    print(classification_report(y_true, pred, digits=4))

    return {
        'model': model_name,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'threshold': threshold,
        'f1': best_f1,
        'precision': precision,
        'recall': recall
    }

# 5. Train Models

**5.1 CatBoost**

In [ ]:
!pip -q install lightgbm xgboost catboost interpret


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.1/780.1 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 22.9 MB/s eta 0:00:00


In [91]:
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from interpret.glassbox import ExplainableBoostingClassifier

In [81]:
CATEGORICAL_COLS = [
    "driver_gender",
    "driver_occupation",
    "vehicle_type",
    "vehicle_segment",
    "fuel_type",
    "vehicle_usage_type",
    "risk_exposure_proxy",
    "registration_district",
    "parking_type",
    "driver_age_band",
    "vehicle_age_band"
]

In [82]:
cat_indices = [X_train.columns.get_loc(col) for col in CATEGORICAL_COLS]

print("\n" + "="*80)
print("READY FOR MODEL TRAINING")
print("="*80)
print(f"Categorical feature indices: {cat_indices}")
print(f"Total features: {len(RISK_FEATURES)}")
print(f"Encoded features: {X_train_enc.shape[1]}")


READY FOR MODEL TRAINING
Categorical feature indices: [2, 3, 9, 10, 12, 16, 17, 18, 19, 1, 14]
Total features: 22
Encoded features: 55


In [84]:
print("\n" + "="*80)
print("TRAINING CATBOOST")
print("="*80)

catboost_model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.05,
    depth=8,
    loss_function="Logloss",
    eval_metric="AUC",
    class_weights=[class_weight_dict[0], class_weight_dict[1]],
    random_seed=42,
    verbose=200
)

catboost_model.fit(
    X_train, y_train,
    cat_features=CATEGORICAL_COLS,
    eval_set=(X_val, y_val),
    use_best_model=True
)


val_proba_cat = catboost_model.predict_proba(X_val)[:, 1]
results_cat = evaluate_model(y_val, val_proba_cat, model_name="CatBoost (Validation)")


TRAINING CATBOOST
0:	test: 0.6293783	best: 0.6293783 (0)	total: 184ms	remaining: 6m 8s
200:	test: 0.6498800	best: 0.6508532 (74)	total: 29.6s	remaining: 4m 24s
400:	test: 0.6450114	best: 0.6508532 (74)	total: 1m 2s	remaining: 4m 11s
600:	test: 0.6387139	best: 0.6508532 (74)	total: 1m 37s	remaining: 3m 46s
800:	test: 0.6328818	best: 0.6508532 (74)	total: 2m 13s	remaining: 3m 20s
1000:	test: 0.6277204	best: 0.6508532 (74)	total: 2m 47s	remaining: 2m 47s
1200:	test: 0.6226610	best: 0.6508532 (74)	total: 3m 23s	remaining: 2m 15s
1400:	test: 0.6208325	best: 0.6508532 (74)	total: 4m 1s	remaining: 1m 43s
1600:	test: 0.6167848	best: 0.6508532 (74)	total: 4m 52s	remaining: 1m 12s
1800:	test: 0.6133260	best: 0.6508532 (74)	total: 5m 27s	remaining: 36.2s
1999:	test: 0.6105178	best: 0.6508532 (74)	total: 6m 3s	remaining: 0us

bestTest = 0.6508532221
bestIteration = 74

Shrink model to first 75 iterations.

CatBoost (Validation) - EVALUATION RESULTS
ROC-AUC:           0.6509
PR-AUC:            0.3

In [85]:
test_proba_cat = catboost_model.predict_proba(X_test)[:, 1]
test_results_cat = evaluate_model(
    y_test, test_proba_cat,
    threshold=results_cat['threshold'],
    model_name="CatBoost (Test)"
)


CatBoost (Test) - EVALUATION RESULTS
ROC-AUC:           0.6469
PR-AUC:            0.3659
Optimal Threshold: 0.480
F1-Score:          0.4387
Precision:         0.3248
Recall:            0.6756

Confusion Matrix:
[[3590 3160]
 [ 730 1520]]

Classification Report:
              precision    recall  f1-score   support

           0     0.8310    0.5319    0.6486      6750
           1     0.3248    0.6756    0.4387      2250

    accuracy                         0.5678      9000
   macro avg     0.5779    0.6037    0.5436      9000
weighted avg     0.7045    0.5678    0.5961      9000



**5.2 LightGBM**

In [86]:
print("\n" + "="*80)
print("TRAINING LIGHTGBM")
print("="*80)

# Convert to DataFrames with proper column names
feature_names = preprocessor.get_feature_names_out()
X_train_df = pd.DataFrame(X_train_enc, columns=feature_names)
X_val_df = pd.DataFrame(X_val_enc, columns=feature_names)
X_test_df = pd.DataFrame(X_test_enc, columns=feature_names)

lgbm_model = lgb.LGBMClassifier(
    n_estimators=8000,
    learning_rate=0.01,
    min_child_samples=80,
    num_leaves=63,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=2.0,
    class_weight=class_weight_dict,
    random_state=42,
    n_jobs=-1
)

lgbm_model.fit(
    X_train_df, y_train,
    eval_set=[(X_val_df, y_val)],
    eval_metric="auc",
    callbacks=[lgb.early_stopping(100)]
)

# Evaluate
val_proba_lgbm = lgbm_model.predict_proba(X_val_df)[:, 1]
results_lgbm = evaluate_model(y_val, val_proba_lgbm, model_name="LightGBM (Validation)")


TRAINING LIGHTGBM
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 10503, number of negative: 31497
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.058898 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 274
[LightGBM] [Info] Number of data points in the train set: 42000, number of used features: 55
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[93]	valid_0's auc: 0.648884	valid_0's binary_logloss: 0.663082

LightGBM (Validation) - EVALUATION RESULTS
ROC-AUC:           0.6489
PR-AUC:            0.3699
Optimal T

In [87]:
test_proba_lgbm = lgbm_model.predict_proba(X_test_df)[:, 1]
test_results_lgbm = evaluate_model(
    y_test, test_proba_lgbm,
    threshold=results_lgbm['threshold'],
    model_name="LightGBM (Test)"
)


LightGBM (Test) - EVALUATION RESULTS
ROC-AUC:           0.6419
PR-AUC:            0.3569
Optimal Threshold: 0.490
F1-Score:          0.4365
Precision:         0.3237
Recall:            0.6702

Confusion Matrix:
[[3599 3151]
 [ 742 1508]]

Classification Report:
              precision    recall  f1-score   support

           0     0.8291    0.5332    0.6490      6750
           1     0.3237    0.6702    0.4365      2250

    accuracy                         0.5674      9000
   macro avg     0.5764    0.6017    0.5428      9000
weighted avg     0.7027    0.5674    0.5959      9000



**5.3 XGBoost**

In [88]:
print("\n" + "="*80)
print("TRAINING XGBOOST")
print("="*80)

# Create DMatrix objects
dtrain = xgb.DMatrix(X_train_df.values, label=y_train.values)
dval = xgb.DMatrix(X_val_df.values, label=y_val.values)
dtest = xgb.DMatrix(X_test_df.values, label=y_test.values)

# Calculate scale_pos_weight
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
scale_pos_weight = neg_count / pos_count

xgb_params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "learning_rate": 0.03,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "lambda": 1.0,
    "scale_pos_weight": scale_pos_weight,
    "seed": 42,
    "tree_method": "hist"
}

xgb_model = xgb.train(
    params=xgb_params,
    dtrain=dtrain,
    num_boost_round=3000,
    evals=[(dval, "val")],
    early_stopping_rounds=100,
    verbose_eval=100
)

# Evaluate
val_proba_xgb = xgb_model.predict(dval)
results_xgb = evaluate_model(y_val, val_proba_xgb, model_name="XGBoost (Validation)")


TRAINING XGBOOST
[0]	val-auc:0.63154
[100]	val-auc:0.64675
[127]	val-auc:0.64540

XGBoost (Validation) - EVALUATION RESULTS
ROC-AUC:           0.6454
PR-AUC:            0.3690
Optimal Threshold: 0.410
F1-Score:          0.4411
Precision:         0.3014
Recall:            0.8223

Confusion Matrix:
[[2458 4291]
 [ 400 1851]]

Classification Report:
              precision    recall  f1-score   support

           0     0.8600    0.3642    0.5117      6749
           1     0.3014    0.8223    0.4411      2251

    accuracy                         0.4788      9000
   macro avg     0.5807    0.5933    0.4764      9000
weighted avg     0.7203    0.4788    0.4940      9000



In [89]:
test_proba_xgb = xgb_model.predict(dtest)
test_results_xgb = evaluate_model(
    y_test, test_proba_xgb,
    threshold=results_xgb['threshold'],
    model_name="XGBoost (Test)"
)


XGBoost (Test) - EVALUATION RESULTS
ROC-AUC:           0.6449
PR-AUC:            0.3611
Optimal Threshold: 0.410
F1-Score:          0.4388
Precision:         0.2994
Recall:            0.8213

Confusion Matrix:
[[2425 4325]
 [ 402 1848]]

Classification Report:
              precision    recall  f1-score   support

           0     0.8578    0.3593    0.5064      6750
           1     0.2994    0.8213    0.4388      2250

    accuracy                         0.4748      9000
   macro avg     0.5786    0.5903    0.4726      9000
weighted avg     0.7182    0.4748    0.4895      9000



**5.4 EBM** - EXPLAINABLE BOOSTING MACHINE

In [92]:
print("\n" + "="*80)
print("TRAINING EBM")
print("="*80)

ebm_model = ExplainableBoostingClassifier(
    random_state=42,
    max_bins=256,
    interactions=10,
    n_jobs=-1
)

ebm_model.fit(X_train_enc, y_train)

# Evaluate
val_proba_ebm = ebm_model.predict_proba(X_val_enc)[:, 1]
results_ebm = evaluate_model(y_val, val_proba_ebm, model_name="EBM (Validation)")


TRAINING EBM

EBM (Validation) - EVALUATION RESULTS
ROC-AUC:           0.6488
PR-AUC:            0.3770
Optimal Threshold: 0.190
F1-Score:          0.4417
Precision:         0.3046
Recall:            0.8032

Confusion Matrix:
[[2621 4128]
 [ 443 1808]]

Classification Report:
              precision    recall  f1-score   support

           0     0.8554    0.3884    0.5342      6749
           1     0.3046    0.8032    0.4417      2251

    accuracy                         0.4921      9000
   macro avg     0.5800    0.5958    0.4879      9000
weighted avg     0.7176    0.4921    0.5111      9000



In [93]:
test_proba_ebm = ebm_model.predict_proba(X_test_enc)[:, 1]
test_results_ebm = evaluate_model(
    y_test, test_proba_ebm,
    threshold=results_ebm['threshold'],
    model_name="EBM (Test)"
)


EBM (Test) - EVALUATION RESULTS
ROC-AUC:           0.6501
PR-AUC:            0.3676
Optimal Threshold: 0.190
F1-Score:          0.4399
Precision:         0.3023
Recall:            0.8071

Confusion Matrix:
[[2559 4191]
 [ 434 1816]]

Classification Report:
              precision    recall  f1-score   support

           0     0.8550    0.3791    0.5253      6750
           1     0.3023    0.8071    0.4399      2250

    accuracy                         0.4861      9000
   macro avg     0.5787    0.5931    0.4826      9000
weighted avg     0.7168    0.4861    0.5039      9000



# Model Comparison

In [94]:
print("\n" + "="*80)
print("MODEL COMPARISON SUMMARY")
print("="*80)

comparison_df = pd.DataFrame([
    test_results_cat,
    test_results_lgbm,
    test_results_xgb,
    test_results_ebm
])

print(comparison_df.to_string(index=False))


MODEL COMPARISON SUMMARY
          model  roc_auc   pr_auc  threshold       f1  precision   recall
CatBoost (Test) 0.646869 0.365857       0.48 0.438672   0.324786 0.675556
LightGBM (Test) 0.641874 0.356901       0.49 0.436532   0.323675 0.670222
 XGBoost (Test) 0.644880 0.361074       0.41 0.438799   0.299368 0.821333
     EBM (Test) 0.650085 0.367607       0.19 0.439869   0.302314 0.807111


In [98]:
best_model = comparison_df.loc[comparison_df['roc_auc'].idxmax(), 'model']
print(f"\n Best model by ROC-AUC: {best_model}")


 Best model by ROC-AUC: EBM (Test)


Risk Scores

In [96]:
if 'CatBoost' in best_model:
    best_proba = test_proba_cat
elif 'LightGBM' in best_model:
    best_proba = test_proba_lgbm
elif 'XGBoost' in best_model:
    best_proba = test_proba_xgb
else:
    best_proba = test_proba_ebm

risk_scores = (best_proba * 100).round().astype(int)

In [97]:
results_df = pd.DataFrame({
    'actual_claim': y_test.values,
    'risk_probability': best_proba,
    'risk_score': risk_scores,
    'predicted_claim': (best_proba >= comparison_df.loc[comparison_df['model'] == best_model, 'threshold'].values[0]).astype(int)
})

print("\n" + "="*80)
print("SAMPLE RISK SCORES")
print("="*80)
print(results_df.head(20))

print(f"\nRisk Score Distribution:")
print(results_df['risk_score'].describe())


SAMPLE RISK SCORES
    actual_claim  risk_probability  risk_score  predicted_claim
0              0          0.354452          35                1
1              1          0.382337          38                1
2              0          0.108835          11                0
3              0          0.175040          18                0
4              1          0.260262          26                1
5              0          0.130167          13                0
6              0          0.215285          22                1
7              1          0.454407          45                1
8              1          0.276781          28                1
9              0          0.353922          35                1
10             0          0.162039          16                0
11             0          0.351124          35                1
12             0          0.497520          50                1
13             0          0.179764          18                0
14             0    